In [1]:
# From Python
# It requires OpenCV installed for Python
import sys
import cv2
import os
from sys import platform
import argparse
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
import pandas as pd

%matplotlib inline

try:
        from openpose import pyopenpose as op
except:
        raise Exception('Error: OpenPose library could not be found. Did you enable `BUILD_PYTHON` in CMake and have this Python script in the right folder?')

In [2]:
def set_params():

        params = dict()
        params["logging_level"] = 3
        params["output_resolution"] = "-1x-1"
        params["net_resolution"] = "-1x368"
        params["model_pose"] = "BODY_25"
        params["alpha_pose"] = 0.6
        params["scale_gap"] = 0.3
        params["scale_number"] = 1
        params["render_threshold"] = 0.05
        params["display"] = 0
        # If GPU version is built, and multiple GPUs are available, set the ID here
        params["num_gpu_start"] = 0
        params["disable_blending"] = False
        # Ensure you point to the correct path where models are located
        params["model_folder"] = "/home/suprnrdy/workspace/openpose/models/"
        return params

def getRectangle(keypoint, threshold):
    #look for min-x, max-x, min-y, max-y
    minx = min(a for (a, b, c) in keypoint if c > threshold)
    maxx = max(a for (a, b, c) in keypoint if c > threshold)
    miny = min(b for (a, b, c) in keypoint if c > threshold)
    maxy = max(b for (a, b, c) in keypoint if c > threshold)
    return [minx, maxx, miny, maxy]

def getArea(box):
    w = box[1] - box[0]
    h = box[3] - box[2]
    return w * h

def getBiggestPerson(keypoints, threshold):
    #loop through each person
    areas = []
    for i in keypoints:
         #calculate rectangle based on min/max of each
        rectangle = getRectangle(i, threshold)
        areas.append(getArea(rectangle))
    #compare all rectangles and return the id of person number
    return np.argmax(areas)

In [3]:
outfile_path = '/home/suprnrdy/workspace/weightlift/cleanandjerk.csv'

In [4]:
data, input_width, input_height, threshold, frame_number = [], 640, 640, 0.1, 0
input_source = "./weightlift-trim.mp4"
cap = cv2.VideoCapture(input_source)

In [5]:
# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(set_params())
opWrapper.start()


In [6]:
frame_number = 0
keypoints = []
keypointSingle = []

In [7]:
while True:
    ret, img = cap.read()
    if not ret: 
        break
        
    # Process Image
    datum = op.Datum()
    datum.cvInputData = img
    opWrapper.emplaceAndPop([datum])

    # Display Image
    biggestPerson = getBiggestPerson(datum.poseKeypoints, threshold)
    keypoints = datum.poseKeypoints[biggestPerson]
#     print(keypoints)
    keypointSingle.append(keypoints)    
#     plt.figure(1); plt.clf()
#     plt.imshow(cv2.cvtColor(datum.cvOutputData, cv2.COLOR_BGR2RGB))
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     plt.title('my picture')
#     plt.pause(1)

    frame_number += 1
    # use this break statement to check your data before processing the whole video
#     if frame_number == 5: 
#         break    
cap.release()
# cv2.destroyAllWindows()

In [8]:
kpnumpy = np.array(keypointSingle)

In [9]:
df = pd.DataFrame(list(map(np.ravel, keypointSingle)))
df.shape
df.to_csv('cleanandjerk.csv', index=False)

In [10]:
# This converts the flattened dataframe back to (n,25,3) array with n being the number of 
# frames, 25 being the pose points, and 3 being x,y,probability for each pose joint.
df2 = pd.read_csv('cleanandjerk.csv')
abc = np.array(df2.values.reshape(df2.shape[0],25,3))
abc.shape

(1305, 25, 3)

In [15]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,268.629242,146.692337,0.903268,339.974487,242.581085,0.768271,298.257446,244.255966,0.798398,301.761566,...,0.148456,298.246552,583.777405,0.489597,299.932800,578.593933,0.448608,397.553467,564.626160,0.445479
1,268.664062,145.046204,0.904204,339.973175,242.562088,0.766197,298.259491,244.226593,0.798119,301.734375,...,0.147836,298.247589,583.772644,0.483101,299.933472,578.594727,0.441915,397.560425,564.618652,0.446252
2,270.280884,145.004288,0.912309,339.967926,242.526230,0.760687,298.252686,244.210571,0.799152,301.735138,...,0.145986,298.251709,583.775391,0.482812,299.934662,578.593079,0.441772,397.557922,564.600159,0.458429
3,270.340424,144.923492,0.907471,339.977051,242.550507,0.765996,298.254791,244.193161,0.798388,301.754089,...,0.145027,298.236145,583.770752,0.483230,298.273773,578.608948,0.444314,397.530884,562.867310,0.481085
4,270.396088,143.289993,0.905794,339.989227,242.566101,0.766802,298.263855,242.564377,0.797888,301.765503,...,0.145475,298.219849,583.772400,0.493106,298.266083,578.609802,0.454859,397.492310,562.895447,0.484242


In [ ]:
# poseModel = op.PoseModel.BODY_25
# print(op.getPoseBodyPartMapping(poseModel))
# print(op.getPoseNumberBodyParts(poseModel))
# print(op.getPosePartPairs(poseModel))
# print(op.getPoseMapIndex(poseModel))

In [ ]:
# Get output for full video into one dataframe
# save dataframe as CSV
# Loop through each joint x and then y and smooth 
# save smoothed video output
# Compare with final video